In [1]:
import pandas as pd
import numpy as np
import spacy
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import gensim.downloader as api

In [4]:
fake = pd.read_csv("/content/drive/MyDrive/NLP PROJECT/DATASET/Fake.csv")
true = pd.read_csv("/content/drive/MyDrive/NLP PROJECT/DATASET/True.csv")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
fake['category']=1
true['category']=0

df=pd.concat([fake,true]).reset_index(drop=True)

In [6]:
# Separate the majority and minority classes
df_majority = df[df['category'] == 1]
df_minority = df[df['category'] == 0]

# Sample 3000 data points from each class
df_majority_sampled = df_majority.sample(n=3000, random_state=42)
df_minority_sampled = df_minority.sample(n=3000, random_state=42)

# Combine the sampled data
df_balanced = pd.concat([df_majority_sampled, df_minority_sampled])

# Shuffle the dataframe
df= df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the balanced dataset if needed
# df_balanced.to_csv('balanced_dataset.csv', index=False)

df_majority = df[df['category'] == 1]
df_minority = df[df['category'] == 0]

# Undersample the majority class
df_majority_undersampled = df_majority.sample(len(df_minority), random_state=42)

# Combine the undersampled majority class with the minority class
df = pd.concat([df_majority_undersampled, df_minority])

In [7]:
df=df[['text','category']]

In [8]:
# Load SpaCy model and define stop words and punctuations
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

In [9]:
# Load pre-trained Word2Vec embeddings
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [10]:
# Function to tokenize and preprocess text
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower().strip() for word in doc]

    # Removing stop words and punctuations
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

    # Return preprocessed list of tokens
    return mytokens

In [11]:
# Apply tokenization to each row in 'data' column
df['data'] = df['text'].apply(spacy_tokenizer)

In [12]:
# Generate word vectors for each document
df['vec'] = df['data'].apply(lambda x: np.mean([wv[token] for token in x if token in wv] or [np.zeros(wv.vector_size)], axis=0))

In [13]:
# Prepare feature matrix and target vector
X = np.vstack(df['vec'])
y = df['category']

In [14]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [15]:
# Create a pipeline with a Decision Tree classifier
model_pipeline_dt = Pipeline([
    ('dt', DecisionTreeClassifier(random_state=42))
])


In [16]:
# Train the model
model_pipeline_dt.fit(X_train, y_train)

Pipeline(steps=[('dt', DecisionTreeClassifier(random_state=42))])

In [17]:
# Predict on the test set
y_pred_dt = model_pipeline_dt.predict(X_test)



In [18]:
accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt, average='weighted')
recall_dt = recall_score(y_test, y_pred_dt, average='weighted')
f1_score_dt = f1_score(y_test, y_pred_dt, average='weighted')
classification_report_dt = classification_report(y_test, y_pred_dt)


In [19]:
# Print evaluation metrics
print("Accuracy:", accuracy_dt)
print("Precision:", precision_dt)
print("Recall:", recall_dt)
print("F1 Score:", f1_score_dt)
print("Evaluation Metrics for Decision Tree Model")
print("------------------------------------------------")
print(classification_report_dt)

Accuracy: 0.8141666666666667
Precision: 0.8141869111280876
Recall: 0.8141666666666667
F1 Score: 0.8141717002435239
Evaluation Metrics for Decision Tree Model
------------------------------------------------
              precision    recall  f1-score   support

           0       0.81      0.81      0.81       592
           1       0.82      0.81      0.82       608

    accuracy                           0.81      1200
   macro avg       0.81      0.81      0.81      1200
weighted avg       0.81      0.81      0.81      1200

